In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

In [ ]:
import os
import json
import operator
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter

from params import *
from utils.logger import Config, upload_to_kaggle

In [ ]:
from data.preparation import *
from data.processing import *
from data.post_processing import *

from utils.plot import *
from utils.metric import *
from inference.main import blend

## Experiment

In [ ]:
EXP_FOLDERS = [
    LOG_PATH + "2022-04-14/0/",   # 60. deberta-xlarge  soft pl3 x8 - 0.8911
#     LOG_PATH + "2022-04-13/1/",   # 60. deberta-xlarge  soft pl3 x8 - 0.8908
    
    LOG_PATH + "2022-04-10/0/",   # 54. deberta-v3-base soft pl3 - 0.891
    LOG_PATH + "2022-04-10/3/",   # 57. roberta-large soft pl3 - 0.8908
    LOG_PATH + "2022-04-11/1/",   # 58. deberta-v3-large soft pl3 - 0.8921
    LOG_PATH + "2022-04-12/3/",   # 59. deberta-large soft pl3 - 0.8919
]

WEIGHTS = [0, 0, 0, 1, 1]

In [ ]:
# upload_to_kaggle(EXP_FOLDERS, DATASETS_DIR + "weights_base/", "NBME Weights base")

## Load

In [ ]:
config = Config(json.load(open(EXP_FOLDERS[0] + "config.json", 'r')))

try:
    _ = config.lower
except AttributeError:
    config.lower = True

In [ ]:
df = load_and_prepare(root=DATA_PATH, lower=config.lower)

In [ ]:
folds = pd.read_csv(config.folds_file)
df = df.merge(folds, how="left", on=["case_num", "pn_num"])

In [ ]:
pred_oof = blend(
    [np.load(f + "pred_oof.npy", allow_pickle=True).tolist() for f in EXP_FOLDERS],
    weights=WEIGHTS
)

# pred_oof = np.load('../output/pred_oof.npy', allow_pickle=True)


df["probs"] = pred_oof
# df["probs1"] = pred_oof
# pred_oof = np.load(EXP_FOLDERS[0] + "pred_oof.npy", allow_pickle=True)

In [ ]:
PROB_THRESHOLD = 0.5


# PROB_THRESHOLDS = [0.4, 0.6, 0.4, 0.6, 0.8, 0.5, 0.6, 0.6, 0.5, 0.5]
# PROB_THRESHOLDS = [0.34, 0.33, 0.45, 0.6, 0.79, 0.47, 0.55, 0.61, 0.75, 0.3]
# PROB_THRESHOLDS = [0.4, 0.4, 0.5, 0.5, 0.6, 0.5, 0.5, 0.5, 0.6, 0.4]

In [ ]:
# df['probs'] = pred_oof
df['preds'] = preds_to_labels(df["probs"].values.tolist(), threshold=PROB_THRESHOLD)
# df['preds'] = preds_to_labels_advanced(pred_oof, df['case_num'].values, thresholds=PROB_THRESHOLDS)

df['preds_pp'] = df.apply(lambda x: post_process_spaces(x['preds'], x['clean_text']), 1)

In [ ]:
for i in range(5):
    df_ = df[df['fold'] == i].reset_index()
    cv_f1 = micro_f1(df_['preds_pp'], df_['target'])
    print(f"-> Fold #{i} score : {cv_f1:.4f}")
#     break

cv_f1 = micro_f1(df['preds_pp'], df['target'])
print(f"\n-> CV score: {cv_f1:.4f}")